<a href="https://colab.research.google.com/github/chicocasanova/mvp_sprint_2/blob/main/MVP_Engenharia_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MVP - Engenharia de dados


##Sobre o database escolhido

Neste MVP estou utilizando um database extraído do Kaggle ("https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023"). Ele consiste em uma tabela com as músicas mais tocadas no ano de 2023 no Spotify e vários atributos relacionados à elas.

###Catálogo de dados

| Atributo                | Descrição                                             |
|-----------------------|-------------------------------------------------------|
| track_name            | nome da música                                        |
| artist(s)_name        | nome do(s) artista(s) da música                       |
| artist_count          | número de artistas contribuindo para a música         |
| released_year         | ano de lançamento da música                           |
| released_month        | mês de lançamento da música                           |
| released_day          | dia do mês de lançamento da música                    |
| in_spotify_playlists  | número de playlists do spotify em que a música está incluída |
| in_spotify_charts     | presença e posição da música nas paradas do spotify   |
| streams               | total de streams no spotify                           |
| in_apple_playlists    | número de playlists do apple music em que a música está incluída |
| in_apple_charts       | presença e posição da música nas paradas do apple music |
| in_deezer_playlists   | número de playlists do deezer em que a música está incluída |
| in_deezer_charts      | presença e posição da música nas paradas do deezer    |
| in_shazam_charts      | presença e posição da música nas paradas do shazam    |
| bpm                   | batidas por minuto, uma medida do tempo da música     |
| key                   | tom da música                                         |
| mode                  | modo da música (maior ou menor)                       |
| danceability          | percentual indicando quão adequada a música é para dançar |
| valence               | positividade do conteúdo musical da música            |
| energy                | nível percebido de energia da música                  |
| acousticness          | quantidade de som acústico na música                  |
| instrumentalness      | quantidade de conteúdo instrumental na música         |
| liveness              | presença de elementos de performance ao vivo          |
| speechiness           | quantidade de palavras faladas na música              |

## Perguntas a serem respondidas
- Existe algum padrão quanto ao tom das músicas que fazem mais sucesso?
- E quanto ao modo (maior ou menor)?
- E quanto ao BPM?
- E quanto ao ano e década de lançamento?
- E quanto ao mês de lançamento?

##Carregando e convertendo a tabela para spark

In [ ]:
#Fazendo os imports
import pandas as pd
from pyspark.sql.functions import *

In [ ]:
#chamando a url do arquivo e convertendo em um dataframe do spark
url="https://raw.githubusercontent.com/chicocasanova/mvp_sprint_2/main/spotify2023.csv"
df=spark.createDataFrame(pd.read_csv(url, encoding="ISO-8859-1"))

In [ ]:
display(df.limit(10))

track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,1.41381703E8,43,263,45.0,10,826.0,125,B,Major,80,89,83,31,0,8,4
LALA,Myke Towers,1,2023,3,23,1474,48,1.33716286E8,48,126,58.0,14,382.0,92,C#,Major,71,61,74,7,0,10,4
vampire,Olivia Rodrigo,1,2023,6,30,1397,113,1.40003974E8,94,207,91.0,14,949.0,138,F,Major,51,32,53,17,0,31,6
Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,8.00840817E8,116,207,125.0,12,548.0,170,A,Major,55,58,72,11,0,11,15
WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,3.03236322E8,84,133,87.0,15,425.0,144,A,Minor,65,23,80,14,63,11,6
Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,1.83706234E8,67,213,88.0,17,946.0,141,C#,Major,92,66,58,19,0,8,24
Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,7.25980112E8,34,222,43.0,13,418.0,148,F,Minor,67,83,76,48,0,8,3
Columbia,Quevedo,1,2023,7,7,714,43,5.8149378E7,25,89,30.0,13,194.0,100,F,Major,67,26,71,37,0,11,4
fukumean,Gunna,1,2023,5,15,1096,83,9.5217315E7,60,210,48.0,11,953.0,130,C#,Minor,85,22,62,12,0,28,9
La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,5.53634067E8,49,110,66.0,13,339.0,170,D,Minor,81,56,48,21,0,8,33


In [ ]:
df.columns

Out[4]: ['track_name',
 'artist(s)_name',
 'artist_count',
 'released_year',
 'released_month',
 'released_day',
 'in_spotify_playlists',
 'in_spotify_charts',
 'streams',
 'in_apple_playlists',
 'in_apple_charts',
 'in_deezer_playlists',
 'in_deezer_charts',
 'in_shazam_charts',
 'bpm',
 'key',
 'mode',
 'danceability_%',
 'valence_%',
 'energy_%',
 'acousticness_%',
 'instrumentalness_%',
 'liveness_%',
 'speechiness_%']

##Iniciando o tratamento dos dados

###Alterando nome dos atributos e mudando o datatype
Alguns atributos tinham % no seus nomes e estavam gerando erro no código, por isso retirei esses caracteres. Outros atributos cujos valores eram todos numéricos estavam com o data type string, os converti para int.

In [ ]:
df = df.withColumnRenamed("danceability_%", "danceability") \
       .withColumnRenamed("valence_%", "valence") \
        .withColumnRenamed("energy_%", "energy") \
        .withColumnRenamed("acousticness_%", "acousticness") \
        .withColumnRenamed("instrumentalness_%", "instrumentalness") \
        .withColumnRenamed("liveness_%", "liveness") \
        .withColumnRenamed("speechiness_%", "speechiness") \
        .withColumnRenamed("artist(s)_name", "artist_name")


In [ ]:
df = df.withColumn("streams", col("streams").cast("double")) \
       .withColumn("in_deezer_playlists", col("in_deezer_playlists").cast("double")) \
       .withColumn("in_shazam_charts", col("in_shazam_charts").cast("double"))

In [ ]:
df.printSchema()


root
 |-- track_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_count: long (nullable = true)
 |-- released_year: long (nullable = true)
 |-- released_month: long (nullable = true)
 |-- released_day: long (nullable = true)
 |-- in_spotify_playlists: long (nullable = true)
 |-- in_spotify_charts: long (nullable = true)
 |-- streams: double (nullable = true)
 |-- in_apple_playlists: long (nullable = true)
 |-- in_apple_charts: long (nullable = true)
 |-- in_deezer_playlists: double (nullable = true)
 |-- in_deezer_charts: long (nullable = true)
 |-- in_shazam_charts: double (nullable = true)
 |-- bpm: long (nullable = true)
 |-- key: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- danceability: long (nullable = true)
 |-- valence: long (nullable = true)
 |-- energy: long (nullable = true)
 |-- acousticness: long (nullable = true)
 |-- instrumentalness: long (nullable = true)
 |-- liveness: long (nullable = true)
 |-- speechiness: lo

###Checando valores nulos
Identifiquei que existem valores nulos nas colunas "in_shazam_charts" e "key". Esses valores nulos não são um grande problema, principalmente os do primeiro atributo, entretanto como uma das minhas perguntas envolve saber se existe um padrão de "key" (tom da música), quando for analisar esse dado devo tomar cuidado para levar em conta esses nulls em eventuais estatísticas. Não os deletarei porque essas linhas servirão para responder às outras perguntas propostas.

In [ ]:
null_counts = df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in df.columns])
display(null_counts)

track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
0,0,0,0,0,0,0,0,1,0,0,79,0,57,0,95,0,0,0,0,0,0,0,0


In [ ]:
# Get unique values in the 'category' column
unique_values = df.select("key").distinct()

# Show the unique values
unique_values.show()

+----+
| key|
+----+
|  C#|
|  D#|
|   F|
|null|
|   E|
|   B|
|  F#|
|   D|
|  G#|
|   A|
|   G|
|  A#|
+----+



###Salvando a tabela em disco

In [ ]:
spark.sql("DROP TABLE IF EXISTS spotify_tbl")

Out[10]: DataFrame[]

In [ ]:
df.coalesce(1).write.parquet("FileStore/tables/spotify.parquet", mode="overwrite")
df.write.mode("overwrite").saveAsTable("default.spotify_tbl")

In [ ]:
df.createOrReplaceTempView("spotify")

In [ ]:
display(df.describe())

summary,track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
count,953,953,953,953,953,953,953,953,952,953,953,874,953,896,953,858,953,953,953,953,953,953,953,953
mean,Infinity,null,1.5561385099685205,2018.2381951731375,6.033578174186778,13.930745015739769,5200.124868835257,12.009443861490032,5.1413742493907565E8,67.8121720881427,51.9087093389297,109.74027459954233,2.666316894018888,51.176339285714285,122.54039874081847,null,null,66.96956977964324,51.43126967471144,64.2791185729276,27.057712486883524,1.5813221406086044,18.213011542497377,10.131164742917104
stddev,NaN,null,0.8930441928452747,11.116218075505879,3.566435132898142,9.201949303162998,7897.608990375098,19.575991636055704,5.668569490388829E8,86.44149293264282,50.630240756979966,178.811406092666,6.035598942337868,126.48785636676816,28.05780150815617,null,null,14.630609874434809,23.48063183989133,16.550525977945235,25.996077310203738,8.4097999265577,13.711223346148799,9.912887609763244
min,'Till I Collapse,(G)I-DLE,1,1930,1,1,31,0,2762.0,0,0,0.0,0,0.0,65,A,Major,23,4,9,0,0,3,2
max,ýýýýýýýýýýýýýýýýýýýýý,teto,8,2023,12,31,52898,147,3.703895074E9,672,275,974.0,58,953.0,206,G#,Minor,96,97,97,97,91,97,64


##Respondendo às perguntas

In [ ]:
display(df.summary())

summary,track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
count,953,953,953,953,953,953,953,953,952,953,953,874,953,896,953,858,953,953,953,953,953,953,953,953
mean,Infinity,null,1.5561385099685205,2018.2381951731375,6.033578174186778,13.930745015739769,5200.124868835257,12.009443861490032,5.1413742493907565E8,67.8121720881427,51.9087093389297,109.74027459954233,2.666316894018888,51.176339285714285,122.54039874081847,null,null,66.96956977964324,51.43126967471144,64.2791185729276,27.057712486883524,1.5813221406086044,18.213011542497377,10.131164742917104
stddev,NaN,null,0.8930441928452747,11.116218075505879,3.566435132898142,9.201949303162998,7897.608990375098,19.575991636055704,5.668569490388829E8,86.44149293264282,50.630240756979966,178.811406092666,6.035598942337868,126.48785636676816,28.05780150815617,null,null,14.630609874434809,23.48063183989133,16.550525977945235,25.996077310203738,8.4097999265577,13.711223346148799,9.912887609763244
min,'Till I Collapse,(G)I-DLE,1,1930,1,1,31,0,2762.0,0,0,0.0,0,0.0,65,A,Major,23,4,9,0,0,3,2
25%,212.0,null,1,2020,3,6,875,0,1.41381703E8,13,7,12.0,0,0.0,100,null,null,57,32,53,6,0,10,4
50%,295.0,null,1,2022,6,13,2224,3,2.90228626E8,34,38,36.0,0,2.0,121,null,null,69,51,66,18,0,12,6
75%,2055.0,null,2,2022,9,22,5542,16,6.73801126E8,88,87,110.0,2,36.0,140,null,null,78,70,77,43,0,24,11
max,ýýýýýýýýýýýýýýýýýýýýý,teto,8,2023,12,31,52898,147,3.703895074E9,672,275,974.0,58,953.0,206,G#,Minor,96,97,97,97,91,97,64


###Existe algum padrão quanto ao tom das músicas que fazem mais sucesso?
Os dados mostram que a maior parte (12%) é composta no tom de C#. Os tons de G, G# e F (entre 10 e 9%), B e D (8%), A e F# (7%), E e A#(6%) aparecem sem nenhuma grande predominância entre si.

Existem 95 entradas com valores nulos nesse atributo, o que não desbalanceia muito o cenário, visto que estes correspondem a 10% da amostragem e as outras porcentagens estão relativamente equilibradas.

O que mais me chama atenção é o fato de não ter nenhum música no tom de C, visto que esse é bastante utilizado por músicos por ser o ponto de partida no estudo formal e por estar na faixa vocal da maioria dos cantores/as. Isso me leva a duvidar da precisão desses dados, gostaria de saber qual o critério que utilizaram para avaliar o tom das músicas. Talvez estejam considerando o tom relativo menor ou maior, mas ainda assim a proporção seria estranha.

In [ ]:
# Agrupar por 'key' e contar as ocorrências
counts = df.groupBy("key").count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("porcentagem", round((col("count") / total_count) * 100, 2))

# Ordenar por contagem em ordem decrescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show()

+----+-----+-----------+
| key|count|porcentagem|
+----+-----+-----------+
|  C#|  120|      12.59|
|   G|   96|      10.07|
|null|   95|       9.97|
|  G#|   91|       9.55|
|   F|   89|       9.34|
|   B|   81|        8.5|
|   D|   81|        8.5|
|   A|   75|       7.87|
|  F#|   73|       7.66|
|   E|   62|       6.51|
|  A#|   57|       5.98|
|  D#|   33|       3.46|
+----+-----+-----------+




###E quanto ao modo (maior ou menor)?
58% é em tom maior e 42% é em tom menor.

A proporção é bastante equilibrada. Isso me surpreendeu positivamente, pois como dizem os professores de música, os tons maiores são "felizes" e os menores "tristes", logo, aparentemente a felicidade e tristeza tem igual importância no hábito auditivo das pessoas, o que achei bonito, ainda mais num cenário onde o mundo está cada vez mais maniqueísta.

In [ ]:
# Agrupar por 'mode' e contar as ocorrências
counts = df.groupBy("mode").count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("porcentagem", round((col("count") / total_count) * 100, 2))

# Ordenar por contagem em ordem decrescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show()

+-----+-----+-----------+
| mode|count|porcentagem|
+-----+-----+-----------+
|Major|  550|      57.71|
|Minor|  403|      42.29|
+-----+-----+-----------+



###E quanto ao BPM?
Existe uma clara predominância de músicas com BPM alto ou muito alto estarem nos top charts.

Não há praticamente nenhuma balada no top 20. Destas, apenas 4.5% tem um andamento lento pra rápido (60 a 80bpm) e nenhuma um muito lento.

In [ ]:
# Agrupar por 'bpm' e contar as ocorrências
counts = df.groupBy("bpm").count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("porcentagem", round((col("count") / total_count) * 100, 2))

# Ordenar por contagem em ordem decrescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show()

+---+-----+-----------+
|bpm|count|porcentagem|
+---+-----+-----------+
|120|   39|       4.09|
|130|   31|       3.25|
|140|   31|       3.25|
| 92|   25|       2.62|
|110|   24|       2.52|
|150|   21|        2.2|
| 90|   21|        2.2|
|122|   19|       1.99|
|105|   19|       1.99|
|125|   18|       1.89|
| 96|   17|       1.78|
| 98|   16|       1.68|
| 95|   16|       1.68|
|170|   16|       1.68|
|100|   16|       1.68|
| 94|   15|       1.57|
|144|   15|       1.57|
|124|   14|       1.47|
|118|   14|       1.47|
| 93|   14|       1.47|
+---+-----+-----------+
only showing top 20 rows



In [ ]:
min_value = df.select(min("bpm")).collect()[0][0]

print(f"O menor valor de BPM é: {min_value}")

O menor valor de BPM é: 65


In [ ]:
filtered_df1 = df.filter((df["bpm"] >= 60) & (df["bpm"] <= 70))
filtered_df2 = df.filter((df["bpm"] >= 70) & (df["bpm"] <= 80))

# Contar o número de valores que estão no intervalo
count_in_range1 = filtered_df1.count()
count_in_range2 = filtered_df2.count()

print(f"O número de músicas entre 60 e 70 BPM é: {count_in_range1}")
print(f"O número de músicas entre 70 e 80 BPM é: {count_in_range2}")


O número de músicas entre 60 e 70 BPM é: 3
O número de músicas entre 70 e 80 BPM é: 40


###E quanto ao ano e década de lançamento?
A maior parte das músicas foi lançada em 2022 (42%), o que faz sentido, visto que apesar do chart ser referente ao ano de 2023, os lançamentos deste ano não tiveram tempo o suficiente para serem ouvidos como os do ano anterior. A música mais antiga é do ano de 1930

Agrupando os dados por décadas, maneira na qual muitas playlists são organizadas e critério que muitos usuários utilizam para escolher o que ouvir, vemos que as músicas desta década correspondem a 77% da lista, enquanto as da década de 2010 tem 16% e o restante menos de 2% cada uma, diminuindo progressivamente e paralelamente ao passar do anos.

Esse dado vai contra a tendência notada nas manifestações culturais da sociedade na qual as década de 80 e 90 estão em alta. Apesar de muitos artistas/compositores contemporâneos presentes nessa lista se inspirarem e replicarem em suas músicas elementos dessa época, os usuários não consomem expressivamente as músicas lançadas nas décadas em questão.

Falando especificamente de 1 outlier, a música mais antiga da lista, constando do ano de 1930, em virtude de seus dados estarem praticamente ilegíveis, assim como outras ocorrências da tabela (talvez por algum erro de encoding na hora de geração desta) não consegui identificar o porque da presença dela nesta lista. Imagino que seja em virtude da presença desta em alguma mídia que se tornou viral em redes como o TikTok, ou então por ter sido parte da soundtrack de alguma série ou filme.

In [ ]:
# Agrupar por 'released_year' e contar as ocorrências
counts = df.groupBy("released_year").count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("porcentagem", round((col("count") / total_count) * 100, 2))

# Ordenar por contagem em ordem decrescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show()

+-------------+-----+-----------+
|released_year|count|porcentagem|
+-------------+-----+-----------+
|         2022|  402|      42.18|
|         2023|  175|      18.36|
|         2021|  119|      12.49|
|         2020|   37|       3.88|
|         2019|   36|       3.78|
|         2017|   23|       2.41|
|         2016|   18|       1.89|
|         2014|   13|       1.36|
|         2013|   13|       1.36|
|         2015|   11|       1.15|
|         2012|   10|       1.05|
|         2018|   10|       1.05|
|         2011|   10|       1.05|
|         2010|    7|       0.73|
|         2002|    6|       0.63|
|         1999|    5|       0.52|
|         2004|    4|       0.42|
|         1984|    4|       0.42|
|         2000|    4|       0.42|
|         1958|    3|       0.31|
+-------------+-----+-----------+
only showing top 20 rows



In [ ]:
counts = df.groupBy((floor(col("released_year") / 10) * 10).alias("década")).count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("percentage", round((col("count") / total_count) * 100, 2))

# Ordenar por década em ordem crescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show(truncate=False)


+------+-----+----------+
|década|count|percentage|
+------+-----+----------+
|2020  |733  |76.92     |
|2010  |151  |15.84     |
|2000  |20   |2.1       |
|1990  |14   |1.47      |
|1980  |12   |1.26      |
|1950  |9    |0.94      |
|1970  |7    |0.73      |
|1960  |4    |0.42      |
|1940  |2    |0.21      |
|1930  |1    |0.1       |
+------+-----+----------+



In [ ]:
%sql
SELECT *
FROM spotify_tbl
WHERE released_year = 1930

track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,9.0598517E7,4,0,14.0,0,0.0,130,F#,Minor,65,49,80,22,4,7,5


###E quanto ao mês de lançamento?
A maior parte das músicas desta lista (14% e 13% respectivamente) foram lançadas em janeiro e março. Existe um grande números de lançamento no período de fim de ano, entre os meses de outubro e dezembro, somando 24%, estes praticamente distribuídos de maneira igual dentre os meses em questão. Os demais meses tem valores relativamente bem distribuídos e não divergindo muito dos demais meses do ano (cada um entre 4% e 9%).

Confesso que não conheço o mercado fonográfico o suficiente para saber como funcionam as sazonalidades e estratégias de lançamento. Talvez determinados períodos, como as férias escolares, spring break americano, temporada de inverno e verão, natal, dentre outros, sejam fatores de grande importância para a escolha do mês de lançamento.

Da mesma maneira, como grande parte dos artistas está na mão de um número pequeno de gravadoras, pode ser que estas tenham um cronograma de espaçamento entre seus lançamentos, para que um não ofusque o outro e para que tenham lançamentos o ano todo, o que faz com que eles sejam tão bem distribuídos pelo ano.

In [ ]:
# Agrupar por 'released_month' e contar as ocorrências
counts = df.groupBy("released_month").count()

# Calcular o número total de linhas
total_count = df.count()

# Adicionar uma nova coluna com a porcentagem
counts_with_percentage = counts.withColumn("porcentagem", round((col("count") / total_count) * 100, 2))

# Ordenar por contagem em ordem decrescente
counts_sorted = counts_with_percentage.orderBy("count", ascending=False)

# Mostrar as contagens ordenadas com porcentagem
counts_sorted.show()

+--------------+-----+-----------+
|released_month|count|porcentagem|
+--------------+-----+-----------+
|             1|  134|      14.06|
|             5|  128|      13.43|
|             6|   86|       9.02|
|             3|   86|       9.02|
|            11|   80|       8.39|
|            12|   75|       7.87|
|            10|   73|       7.66|
|             4|   66|       6.93|
|             7|   62|       6.51|
|             2|   61|        6.4|
|             9|   56|       5.88|
|             8|   46|       4.83|
+--------------+-----+-----------+



##Propostas para melhorias futuras

####Descobrir como os dados referentes à tonalidade foram coletados
Achei muito estranho não haver nenhuma música no tom de C, acredito que possa ter algum erro nos dados deste atributo. Talvez o algoritmo usado seja ineficiente para isto. Gostaria de cruzar essa informação com o sexo do/a vocalista, para saber se isso teria a ver com a extensão vocal. Talvez tenha a ver também com alguma trend de mixagem.

####Extrair mais dados de diferentes atributos
Acho que esse estudo poderia ser mais rico caso soubessemos o idioma das letras, a nacionalidade e ano de estréia do artista/banda. Assim se tornaria possível, por exemplo, analisar o quanto o tempo de carreira de um artista influencia em uma música estar no topchart, ou então qual o sucesso do cada vez maior número de músicas bilíngues, como os reggaetons.

####Resolver os problemas de encoding na hora da coleta dos dados
Existem algumas ocorrências em atributos string onde o valor destes está totalmente ilegível. Tentei resolver analisando e mudando o encoding mas não consegui resolver essa questão.

####Cruzar com databases de outras fontes
Seria interessante cruzar esses dados do spotify com os de músicas mais usadas no TikTok e Instagram, assim como com os outros streamings de música (Deezer, Apple...). Assim poderiam ser criados outros insights bastante interessantes sobre o que faz um trend hit.

##Autoavaliação
Minha base de python ainda é fraca, comecei a estudar há apenas 6 meses, em paralelo com a pós, e preciso urgentemente desenvolvê-la mais. Isso faz com que eu tenha muita dificuldade para codificar e realizar algumas tarefas simples, cujo código final poderia também ser bem melhor. Perdi mais de 2 dias para conseguir importar o banco de dados para o dbfs e apenas o consegui porque um colega que deve ser canonizado, Rafael Puyau, me ajudou através de uma call. Gostaria de ter realizado alguns cruzamentos de dados mas, em função da minha falta de destreza como programador, não consegui em tempo viável.

Tive muitos problemas para começar o projeto pois, apesar de ter estudado bastante sobre, não consegui configurar a AWS e o DataBricks premium de maneira efetiva, o que fez com que tivesse gastos com EC2 e demorasse muito até começar a realizar o trabalho, que só conseguiu ter prosseguimento após a mudança para o ambiente mais amigável do DataBricks Community.

Gostei bastante dos insights que gerei, como tenho bastante conhecimento musical e curiosidade sobre o mercado fonográfico, foi muito prazeroso debruçar-me sobre esse assunto. Aguardo ansiosamente a sprint seguinte para aprender a fazer scraping de dados e gerar meus próprios databases sobre o tema.

